In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

In [3]:
train_dir = 'AugTraining'


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2 # Set validation split
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # This is training data
)

Found 48151 images belonging to 4 classes.


In [6]:
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # This is validation data
)


Found 12036 images belonging to 4 classes.


In [7]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
base_model.trainable = False


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    validation_data=val_generator,
    validation_steps=100,
    epochs=10
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.7597 - loss: 0.5791 - val_accuracy: 0.7947 - val_loss: 0.5322
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 250s 3s/step - accuracy: 0.7984 - loss: 0.5086 - val_accuracy: 0.7459 - val_loss: 0.6838
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.8076 - loss: 0.4686 - val_accuracy: 0.7784 - val_loss: 0.5724
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.8159 - loss: 0.4534 - val_accuracy: 0.7640 - val_loss: 0.5909
Epoch 5/10


C:\Users\kusha\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.8052 - loss: 0.4997 - val_accuracy: 0.7962 - val_loss: 0.5305
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8170 - loss: 0.4653 - val_accuracy: 0.7859 - val_loss: 0.5526
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.8226 - loss: 0.4501 - val_accuracy: 0.7900 - val_loss: 0.5429
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.8402 - loss: 0.4337 - val_accuracy: 0.7590 - val_loss: 0.6046
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.8162 - loss: 0.4505 - val_accuracy: 0.8041 - val_loss: 0.5020
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.8372 - loss: 0.4045 - val_accuracy: 0.8100 - val_loss: 0.5078


In [15]:
val_loss, val_acc = model.evaluate(val_generator)
print(f'Validation Accuracy: {val_acc*100:.2f}%')

377/377 ━━━━━━━━━━━━━━━━━━━━ 389s 1s/step - accuracy: 0.8042 - loss: 0.5032
Validation Accuracy: 80.58%


In [16]:
model.save('MobileNetV2.h5')
